In [29]:
import cv2
from imutils.video import VideoStream
import numpy as np
import pickle
import os
import time
import imutils
import csv
from datetime import date
import datetime
from time import gmtime,strftime

In [30]:
#Loadind face detector
detector = cv2.dnn.readNetFromCaffe('./face_detection_model/deploy.prototxt','./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel')

In [31]:
#image to 128d features converter
embedder = cv2.dnn.readNetFromTorch('./openface_nn4.small2.v1.t7')

In [32]:
#trained face recognition model
recognizer = pickle.loads(open('./output/recognizer1.pickle','rb').read())
label = pickle.loads(open('./output/le1.pickle','rb').read())

In [33]:
def detect_and_predict(frame):
    (h, w) = frame.shape[:2]
    imageblob = cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0),swapRB=False,crop=False)
    detector.setInput(imageblob)
    detections = detector.forward()
    locs = []
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype('int')
            face = frame[startY:endY,startX,endX]
            (fw,fh) = face.shape[:2]
            if fw < 20 or fh <20:
                continue
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vect = embedder.forward()
            pred = recognizer.predict_proba(vect)[0]
            j = np.argmax(pred)
            prob = pred[j]
            name = label.classes_[j]
            locs.append((startX,startY,endX,endY))
    return (locs,pred)
            
            
    
    

In [34]:
now = datetime.datetime.now()
month = now.strftime("%B")
day = now.strftime("%d")
filename = f'{day}-{month}-Attendance.csv'
fields = ['Name','Time']
with open(filename,'w+') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
def markattendance(name):
    now = datetime.datetime.now()
    cutime = now.strftime("%H:%M:%S")
    with open(filename,'r+') as f:
        mydatalist = f.readlines()
        namelist=[]
        for line in mydatalist:
            entry = line.split(',')
            namelist.append(entry[0])
        if name not in namelist:
            if name != 'unknown':
                f.writelines(f'\n{name},{cutime}')
            
    

In [35]:
#vs = cv2.VideoCapture('./naveen.mp4')
vs = VideoStream(0,resolution=400).start()#webcam uncomment this
time.sleep(2.0)
while True:
    #ret,frame = vs.read()
    frame = vs.read()
   # if ret == False:
    #    break
    frame = imutils.resize(frame,width = 600)
    (h, w) = frame.shape[:2]
    imageblob = cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0),swapRB=False,crop=False)
    detector.setInput(imageblob)
    detections = detector.forward()
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype('int')
            face = frame[startY:endY,startX:endX]
            (fw,fh) = face.shape[:2]
            if fw < 20 or fh <20:
                continue
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vect = embedder.forward()
            pred = recognizer.predict_proba(vect)[0]
            j = np.argmax(pred)
            prob = pred[j]
            name = label.classes_[j]
            prob = prob*100
            color = (0,255,0) if name == 'naveen' else (0,0,255) 
            text = "{}: {:.2f}%".format(name,prob)
            cv2.putText(frame, text, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            markattendance(name)    
    cv2.imshow("Face Recog",frame)
    if cv2.waitKey(1) == 27:#press esc to exit
            break
cv2.destroyAllWindows()
vs.stop() #if webcam uncomment this
#vs.release()

    